In [1]:
import os
import glob
import tensorflow as tf
import numpy as np
import dask
import dask.array as da
from PIL import Image
from skimage.transform import resize
from matplotlib import pyplot as plt
import scipy
%matplotlib inline
np.set_printoptions(threshold=10000, linewidth=32)

In [2]:
y = np.asarray(sorted(glob.glob("/home/skyolia/JupyterProjects/data/BASE-CYTO/images etiquetées/*.bmp")))
x = np.asarray(sorted(glob.glob("/home/skyolia/JupyterProjects/data/BASE-CYTO/images originales/*.bmp")))

In [3]:
train = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
test = [70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
 80, 81, 82, 83, 84, 85, 86]

In [3]:
rnd = np.random.choice(87, 61, False)
#train_x, train_y = x[train_idx], y[train_idx]

splitter = np.isin(np.arange(87), rnd, True)
train, test = np.where(splitter)[0], np.where(~splitter)[0]

np.sum(splitter), train, test

(61, array([ 0,  1,  2,  3,  4,  5,
        10, 11, 12, 13, 15, 16,
        17, 19, 20, 22, 23, 24,
        25, 26, 27, 28, 29, 32,
        34, 35, 36, 37, 38, 39,
        40, 41, 42, 43, 44, 45,
        46, 49, 50, 51, 54, 55,
        58, 59, 65, 66, 67, 68,
        69, 70, 72, 73, 74, 75,
        77, 78, 80, 82, 83, 84,
        85]), array([ 6,  7,  8,  9, 14, 18,
        21, 30, 31, 33, 47, 48,
        52, 53, 56, 57, 60, 61,
        62, 63, 64, 71, 76, 79,
        81, 86]))

In [4]:
def padding(obj, i):
    pad_size = np.asarray(obj) - np.asarray(i.shape)
    padded = np.pad(i, ((0, pad_size[0]), (0, pad_size[1]), (0, pad_size[2])), mode='edge')
    return padded

def extract_patch_single_image(i, crop_size):
    crops = []
    height, width = i.shape[0], i.shape[1]
    for h in range(0, height, 25):
        for w in range(0, width, 25):
            image = i[h:h+crop_size[0], w:w+crop_size[1], :]
            image = padding((crop_size[0], crop_size[1], image.shape[2]), image) if image.shape[0:2] != crop_size else image
            crops.append(image)
    return np.asarray(crops)

def load_save(path, mode, crop_size, new_shape=None):
    #image = Image.open(path).convert(mode)
    image = scipy.ndimage.imread(path, mode=mode)
    image = np.asarray(image)#/255.0
    image = image[:, :, np.newaxis] if image.ndim == 2 else image
    image = extract_patch_single_image(image, crop_size) if crop_size else image
    image = resize(image, new_shape, preserve_range = True) if new_shape else image
    return image#.astype(int)

def data_processing(data_paths, crop_size, new_shape, mode, chunk, stack=True):
    x = []
    for i in data_paths:
        image = dask.delayed(load_save)(i, mode, crop_size, new_shape=None)
        x.append(da.from_delayed(image, new_shape, dtype=int))
    res = da.stack(x, axis=0).rechunk(chunk) if stack else da.concatenate(x, axis=0).rechunk(chunk)
    return res

train_x = data_processing(x[train], (32, 32), (1271, 32, 32, 3), 'RGB', (15000, 32, 32, 3), False)
train_y = data_processing(y[train], (32, 32), (1271, 32, 32, 1), 'L', (15000, 32, 32, 1), False)
#colored_train_y = data_processing(masks[train], (32, 32), (7400, 32, 32, 3), 'RGB', (10000, 32, 32, 3), False)
test_x = data_processing(x[test], None, (768, 1024, 3), 'RGB', (1, 768, 1024, 3))
test_y = data_processing(y[test], None, (768, 1024, 1), 'L', (1, 768, 1024, 1))
colored_test_y = data_processing(y[test], None, (768, 1024, 3), 'RGB', (1, 768, 1024, 3))
train_x, train_y, test_x, test_y, colored_test_y

(dask.array<rechunk-merge, shape=(88970, 32, 32, 3), dtype=int64, chunksize=(15000, 32, 32, 3)>,
 dask.array<rechunk-merge, shape=(88970, 32, 32, 1), dtype=int64, chunksize=(15000, 32, 32, 1)>,
 dask.array<stack, shape=(17, 768, 1024, 3), dtype=int64, chunksize=(1, 768, 1024, 3)>,
 dask.array<stack, shape=(17, 768, 1024, 1), dtype=int64, chunksize=(1, 768, 1024, 1)>,
 dask.array<stack, shape=(17, 768, 1024, 3), dtype=int64, chunksize=(1, 768, 1024, 3)>)

In [5]:
#train_y = da.where(train_y==118, 117, train_y)
%time da.unique(train_y).compute(), da.unique(test_y).compute()

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


CPU times: user 3.19 s, sys: 2.07 s, total: 5.26 s
Wall time: 9.05 s


(array([  0,  59, 117, 177],
       dtype=uint8), array([  0,  59, 117, 177],
       dtype=uint8))

In [5]:
def one_hot_labels_convertion(y):
    res = []
    for i in da.unique(y).compute():
        res.append(da.where(y==i, 1., 0))
    return da.concatenate(res, axis=3)

train_y = one_hot_labels_convertion(train_y)
test_y = one_hot_labels_convertion(test_y)
train_x, test_x = train_x/255., test_x/255.
train_x, train_y, test_x, test_y

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


(dask.array<truediv, shape=(88970, 32, 32, 3), dtype=float64, chunksize=(15000, 32, 32, 3)>,
 dask.array<concatenate, shape=(88970, 32, 32, 4), dtype=float64, chunksize=(15000, 32, 32, 1)>,
 dask.array<truediv, shape=(17, 768, 1024, 3), dtype=float64, chunksize=(1, 768, 1024, 3)>,
 dask.array<concatenate, shape=(17, 768, 1024, 4), dtype=float64, chunksize=(1, 768, 1024, 1)>)

In [6]:
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(input_shape, n_filters, dropout=0.1):
    image_input = tf.keras.Input(shape=input_shape, name='input_layer')
    
    conv_1 = build_block(image_input, n_filters) #(32, 32, 16)
    conv_2 = build_block(conv_1, n_filters) #(32, 32, 16)
    pool_1 = tf.keras.layers.AveragePooling2D(padding='same')(conv_2)#(16, 16, 16)
    drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(drop_1, n_filters * 2)#(16, 16, 32)
    conv_4 = build_block(conv_3, n_filters * 2)#(16, 16, 32)
    pool_2 = tf.keras.layers.AveragePooling2D(padding='same')(conv_4)#(8, 8, 32)
    drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(drop_2, n_filters * 4)#(8, 8, 64)
    conv_6 = build_block(conv_5, n_filters * 4) #(8, 8, 64)
    pool_3 = tf.keras.layers.AveragePooling2D(padding='same')(conv_6)#(4, 4, 64)
    drop_3 = tf.keras.layers.SpatialDropout2D(dropout)(pool_3)
    
    conv_7 = build_block(drop_3, n_filters * 8)#(4, 4, 128)
    conv_8 = build_block(conv_7, n_filters * 8) #(4, 4, 128)
    
    upsp_1 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_8) #(-1, 8, 8, 128)
    upsp_1 = tf.keras.layers.concatenate([upsp_1, conv_6]) #(-1, 8, 8, 192)
    conv_9 = build_block(upsp_1, n_filters * 4) #(-1, 8, 8, 64)
    conv_10 = build_block(conv_9, n_filters * 4)
    drop_7 = tf.keras.layers.SpatialDropout2D(dropout)(conv_10)
    
    upsp_3 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_7) #(-1, 16, 16, 128)
    upsp_3 = tf.keras.layers.concatenate([upsp_3, conv_4]) #(-1, 16, 16, 192)
    conv_15 = build_block(upsp_3, n_filters * 2) #(-1, 16, 16, 64)
    conv_16 = build_block(conv_15, n_filters * 2)
    drop_7 = tf.keras.layers.SpatialDropout2D(dropout)(conv_16)
    
    upsp_4 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_7) #(-1, 32, 32, 64)
    upsp_4 = tf.keras.layers.concatenate([upsp_4, conv_2])#(-1, 32, 32, 92)
    conv_17 = build_block(upsp_4, n_filters)#(-1, 32, 32, 32)
    conv_18 = build_block(conv_17, n_filters)
    drop_8 = tf.keras.layers.SpatialDropout2D(dropout)(conv_18)
    
    output = tf.keras.layers.Conv2D(4, (1, 1), kernel_initializer='glorot_normal', activation='softmax')(drop_8)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model
        
model = build_unet(input_shape=(None, None, 3), n_filters=16)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 1 432         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 1 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 1 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [7]:
epochs, batch_size, lr, filepath = 1000000, 256, 0.001, "day_1.weights.best.hdf5"
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))

data_gen_args = dict(horizontal_flip=True, vertical_flip=True)#, width_shift_range=0.1, height_shift_range=0.1
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_x, augment=True, seed=seed)
mask_datagen.fit(train_y, augment=True, seed=seed)

image_generator = image_datagen.flow(x=train_x, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(x=train_y, batch_size=batch_size, seed=seed)
train_generator = zip(image_generator, mask_generator)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow(x=test_x, y=test_y, batch_size=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = tf.keras.callbacks.TensorBoard(log_dir=os.getcwd())

opt = tf.keras.optimizers.Adam() # 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.metrics_names)

model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=False,
                    workers=12,
                    shuffle=True,
                    initial_epoch=0,
                    callbacks=[checkpoint, tb])

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


['loss', 'acc']
Epoch 1/1000000
347/348 [============================>.] - ETA: 0s - loss: 66.5215 - acc: 0.9228
Epoch 00001: val_acc improved from -inf to 0.72976, saving model to day_1.weights.best.hdf5
348/348 [==============================] - 76s 218ms/step - loss: 66.3998 - acc: 0.9229 - val_loss: 0.8918 - val_acc: 0.7298
Epoch 2/1000000
347/348 [============================>.] - ETA: 0s - loss: 45.5901 - acc: 0.9413
Epoch 00002: val_acc improved from 0.72976 to 0.94762, saving model to day_1.weights.best.hdf5
348/348 [==============================] - 54s 156ms/step - loss: 45.5132 - acc: 0.9413 - val_loss: 0.1948 - val_acc: 0.9476
Epoch 3/1000000
347/348 [============================>.] - ETA: 0s - loss: 39.3759 - acc: 0.9475
Epoch 00003: val_acc did not improve from 0.94762
348/348 [==============================] - 56s 161ms/step - loss: 39.3189 - acc: 0.9476 - val_loss: 0.5393 - val_acc: 0.7589
Epoch 4/1000000
347/348 [============================>.] - ETA: 0s - loss: 35.991

Epoch 60/1000000
347/348 [============================>.] - ETA: 0s - loss: 20.5164 - acc: 0.9703
Epoch 00060: val_acc did not improve from 0.95736
348/348 [==============================] - 56s 161ms/step - loss: 20.4882 - acc: 0.9703 - val_loss: 0.1743 - val_acc: 0.9488
Epoch 61/1000000
347/348 [============================>.] - ETA: 0s - loss: 20.3236 - acc: 0.9707
Epoch 00061: val_acc did not improve from 0.95736
348/348 [==============================] - 56s 160ms/step - loss: 20.2906 - acc: 0.9708 - val_loss: 0.1376 - val_acc: 0.9557
Epoch 62/1000000
347/348 [============================>.] - ETA: 0s - loss: 20.1692 - acc: 0.9710
Epoch 00062: val_acc did not improve from 0.95736
348/348 [==============================] - 56s 160ms/step - loss: 20.1316 - acc: 0.9711 - val_loss: 0.1742 - val_acc: 0.9411
Epoch 63/1000000
347/348 [============================>.] - ETA: 0s - loss: 20.0620 - acc: 0.9710
Epoch 00063: val_acc did not improve from 0.95736
348/348 [========================

347/348 [============================>.] - ETA: 0s - loss: 18.3371 - acc: 0.9730
Epoch 00090: val_acc did not improve from 0.95736
348/348 [==============================] - 56s 160ms/step - loss: 18.3091 - acc: 0.9731 - val_loss: 0.1661 - val_acc: 0.9444
Epoch 91/1000000
347/348 [============================>.] - ETA: 0s - loss: 18.2297 - acc: 0.9733
Epoch 00091: val_acc did not improve from 0.95736
348/348 [==============================] - 56s 160ms/step - loss: 18.1994 - acc: 0.9733 - val_loss: 0.1396 - val_acc: 0.9555
Epoch 92/1000000
347/348 [============================>.] - ETA: 0s - loss: 18.4262 - acc: 0.9732
Epoch 00092: val_acc did not improve from 0.95736
348/348 [==============================] - 56s 160ms/step - loss: 18.3981 - acc: 0.9732 - val_loss: 0.1612 - val_acc: 0.9458
Epoch 93/1000000
347/348 [============================>.] - ETA: 0s - loss: 18.1361 - acc: 0.9733
Epoch 00093: val_acc did not improve from 0.95736
348/348 [==============================] - 56s 161

Epoch 120/1000000
347/348 [============================>.] - ETA: 0s - loss: 17.0163 - acc: 0.9749
Epoch 00120: val_acc did not improve from 0.95783
348/348 [==============================] - 56s 160ms/step - loss: 16.9878 - acc: 0.9749 - val_loss: 0.1523 - val_acc: 0.9487
Epoch 121/1000000
347/348 [============================>.] - ETA: 0s - loss: 16.9314 - acc: 0.9750
Epoch 00121: val_acc did not improve from 0.95783
348/348 [==============================] - 56s 160ms/step - loss: 16.9143 - acc: 0.9749 - val_loss: 0.1741 - val_acc: 0.9418
Epoch 122/1000000
347/348 [============================>.] - ETA: 0s - loss: 16.8898 - acc: 0.9751
Epoch 00122: val_acc did not improve from 0.95783
348/348 [==============================] - 56s 160ms/step - loss: 16.8622 - acc: 0.9751 - val_loss: 0.2239 - val_acc: 0.9321
Epoch 123/1000000
347/348 [============================>.] - ETA: 0s - loss: 16.9268 - acc: 0.9748
Epoch 00123: val_acc did not improve from 0.95783
348/348 [====================

Epoch 150/1000000
347/348 [============================>.] - ETA: 0s - loss: 15.8457 - acc: 0.9763
Epoch 00150: val_acc did not improve from 0.95977
348/348 [==============================] - 56s 160ms/step - loss: 15.8290 - acc: 0.9763 - val_loss: 0.1635 - val_acc: 0.9477
Epoch 151/1000000
 37/348 [==>...........................] - ETA: 50s - loss: 16.5068 - acc: 0.9754

KeyboardInterrupt: 

In [6]:
#model.save('model.h5')
#del model
loaded_model = tf.keras.models.load_model('model.h5')
scores = loaded_model.evaluate(test_x, test_y, batch_size=1)
scores

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


17/17 [==============================] - 9s 532ms/step


[0.1927107563790153, 0.9401602639871485]

In [7]:
from sklearn.metrics import accuracy_score
pred = tf.keras.utils.to_categorical(np.argmax(loaded_model.predict(x=test_x, batch_size=1), axis=-1), num_classes=4)
accuracy_score(np.argmax(test_y, -1).ravel(), np.argmax(pred, -1).ravel())

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


0.940160265155867